In [ ]:
# !pip install wrds
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
import wrds
db = wrds.Connection()

In [ ]:
db.list_tables(library='crsp')

In [ ]:
# the daily stock file doesn't have ticker (just permno and cusip) nor market
# get list of permnos based on tickers of interest
# print(db.get_table(library='crsp', table='ccmxpf_linktable', obs=10 ).columns)
print(db.get_table(library='crsp', table='ccm_lookup', obs=10 ).columns)
ccm_lookup = db.get_table(library='crsp', table='ccm_lookup')

In [ ]:
sample_firms = (
                ccm_lookup
                    .query("tic in ['AAPL','MSFT','VZ','F'] & lpermno.notnull()", engine='python') 
                    [['gvkey','lpermno','tic']].drop_duplicates()
                )
sample_firms

In [ ]:
# get return data from daily stock file (DSF)
crsp_ret = db.raw_sql('select permno,date,ret,prc,vol,shrout from crsp.dsf where permno in (14593, 65875, 25785, 10107);')


In [ ]:
output_df = (crsp_ret 
     .merge(sample_firms, left_on='permno',right_on='lpermno') # add ticker for students 
     .drop('lpermno',axis=1)                                   # drop duplicative ID
     .assign(date = pd.to_datetime(crsp_ret['date']))          # make "date" a datetime type
     .query('date.dt.year >= 1990 & date.dt.year <= 2020')     # reduce to date range
)
output_df.to_stata('../3firm_ret_1990_2020.dta')